Данная блконот суммирует мой опыт и описывает, как очистить грязные поля валюты и преобразовать их в числовые значения для дальнейшего анализа. 

Так выглядят грязные данные в `Excel`:

![alt text](image/currency.png)

In [1]:
import pandas as pd 

In [66]:
df_orig = pd.read_excel('data/sales_cleanup.xlsx')
df = df_orig.copy()
df

,Customer,Sales
0,Jones Brothers,500
1,Beta Corp,"$1,000.00"
2,Globex Corp,300.1
3,Acme,$750.01
4,Initech,300
5,Hooli,250


In [36]:
df.dtypes

Customer    object
Sales       object
dtype: object

Неудивительно, что столбец `Sales` (Продажи) хранится как `object`. Знаки `$` и `,` - это явные признаки того, что столбец `Sales` не является числовым. Скорее всего, мы захотим провести вычисления со столбцом, поэтому давайте попробуем преобразовать его в число с плавающей точкой.

In [40]:
df['Sales'] = df['Sales'].str.replace(',','')
df['Sales'] = df['Sales'].str.replace('$','')
df['Sales']

0        NaN
1    1000.00
2        NaN
3     750.01
4        NaN
5        NaN
Name: Sales, dtype: object

Я не ожидал этого. По какой-то причине строковые значения были очищены, но другие значения преобразованы в `NaN`. Это большая проблема.

Давайте посмотрим на типы данных в этом наборе:

In [41]:
df = df_orig.copy()
df['Sales'].apply(type)

0      <class 'int'>
1      <class 'str'>
2    <class 'float'>
3      <class 'str'>
4      <class 'int'>
5      <class 'int'>
Name: Sales, dtype: object

Некоторые значения являются числами с плавающей точкой, некоторые - целыми числами, а некоторые - строками. В целом столбец - это `object`.

Мы можем добавить отформатированный столбец, показывающий каждый тип:

In [42]:
df['Sales_type'] = df['Sales'].apply(lambda x : type(x).__name__)
df['Sales_type']

0      int
1      str
2    float
3      str
4      int
5      int
Name: Sales_type, dtype: object

Или вот более компактный способ проверить типы данных в столбце с помощью метода `value_counts()`:

In [43]:
df['Sales'].apply(type).value_counts()

<class 'int'>      3
<class 'str'>      2
<class 'float'>    1
Name: Sales, dtype: int64

In [48]:
def clean_currency(x):
    """ Если значение является строкой, то удаляет символ валюты и разделители, 
    в противном случае - значение является числовым и может быть преобразовано.
    """
    if isinstance(x, str):
        return(float(x.replace('$', '').replace(',', '')))
    return(float(x))

In [49]:
df['Sales'] = df['Sales'].apply(clean_currency)

In [50]:
df['Sales']

0     500.00
1    1000.00
2     300.10
3     750.01
4     300.00
5     250.00
Name: Sales, dtype: float64

In [51]:
df['Sales_type'] = df['Sales'].apply(lambda x : type(x).__name__)

In [52]:
df['Sales_type']

0    float
1    float
2    float
3    float
4    float
5    float
Name: Sales_type, dtype: object

In [56]:
df

,Customer,Sales,Sales_type
0,Jones Brothers,500.00,float
1,Beta Corp,1000.00,float
2,Globex Corp,300.10,float
3,Acme,750.01,float
4,Initech,300.00,float
5,Hooli,250.00,float


In [57]:
 df.dtypes

Customer       object
Sales         float64
Sales_type     object
dtype: object

## Альтернативное решение 

In [62]:
df =df_orig.copy()

In [63]:
df['Sales'] = df['Sales'].replace({'\$': '', ',': ''}, regex=True).astype(float)
df['Sales']

0     500.00
1    1000.00
2     300.10
3     750.01
4     300.00
5     250.00
Name: Sales, dtype: float64

## Резюме

Тип данных `object` обычно используется для хранения строк. Однако можно предположить, что все типы данных в столбце `object` будут строками. Это может быть видно при загрузке грязных данных о валюте, которые могут включать числовые значения с символами, а также целые числа и числа с плавающей точкой.

Вполне возможно, что наивные подходы к очистке непреднамеренно преобразуют числовые значения в NaN.